### 1º Voy a priorizar en primer lugar a los  diseñadores y desarrolladores (lugares con aglomeración de empresas de videojuegos y tecnológicas que hayan facturado más de 1M en su historia, las de videojuegos cumplen el perfil en este caso)

Me quedo de la base de datos Companies con las empresas de videojuegos que han facturado más de 1M, ejecuto el script que escribe el json con la consulta filtrando la base de datos todo con el formato adecuado

In [1]:
from getJsonGames1M import writeJsonOffice
writeJsonOffice()

Hago el mongoimport e importo la colección a la base de datos,indexo el campo position como índice geoespacial. Ahora puedo buscar los lugares donde se aglomeran estas empresas, y buscar la lista de ciudades. El scrip devuelve una lista de ciudades donde hay empresas que a 100km a la redonda tienen más de 27 empresas de videojuegos que han facturado más de 1M, creo que es suficiente para satisfacer a los desarrolladores y a los diseñadores. 

In [2]:
from getCitiesDevNDesigner import citiesDevNDesignerOK
citiesDevNDes = citiesDevNDesignerOK()

In [3]:
print(citiesDevNDes[1])

{'city': 'Brooklyn', 'lat': 40.714952833333335, 'long': -73.9684315}


### 2º En las anteriores ciudades voy a buscar qué tal son las comunicaciones, así cumplo con el requisito de los ejecutivos, para ello ejecuto el script que buscará aeropuertos, estaciones de tren y autobús con la api de google place pasándole el diccionario de las ciudades con sus coordenadas. El criterio va a ser que al menos haya 1 aeropuerto y 10 estaciones de tren o metro en 20 km a la redonda. Es de suponer que todas las ciudades que cumplan el criterio también tendrán buenas comunicaciones por carretera.

In [4]:
from filterCitiesForExecutive import getCities

In [5]:
cities = getCities(citiesDevNDes)

In [6]:
len(cities)

38

### 3º Ahora voy a buscar ciudades con una buena aglomeración de Starbucks, pienso que 10 en 10 km a la redonda es adecuado

In [7]:
from filterCitiesStarbucks import filterStarbucks

In [8]:
cities = list(filter(filterStarbucks,cities))

In [9]:
len(cities)

34

### 4º El siguiente filtro que voy a aplicar a mi lista es uno que satisfaga a las familias, me ha parecido que el criterio de que haya más de 5 lugares con la palabra universidad en 10 km cuadrados es un buen criterio, si hay universidades también hay colegios y guarderías para los niños

In [10]:
from filterCitiesSchool import filterUniversity
cities = list(filter(filterUniversity,cities))


In [11]:
len(cities)

31

### 5º Ahora voy a buscar que haya una buenna aglomeración de bares en la ciudad, si hay bares habrá discotecas y demás establecimientos de ocio nocturno y diurno. 100 bares en 10 km

In [12]:
from filterCitiesBar import filterBar
c = list(filter(filterBar,cities))

In [13]:
len(c)

29

### 6º Ciudades veganas, mas de 55 sitios que contengan 'vegan' en 10km a la redonda

In [14]:
from filterCitiesBar import filterVegan
cities = list(filter(filterVegan,c))
len(cities)

22

### 7. Limpio a mano las ciudades estadounidenses que son distritos.

In [15]:
from cleanDistrites import cleanDistrites
cities = cleanDistrites(cities)
len(cities)

14

In [16]:
cities

[{'city': 'New York', 'lat': 40.735781556250004, 'long': -73.97903525000001},
 {'city': 'London', 'lat': 53.344104, 'long': -6.267494},
 {'city': 'Los Angeles', 'lat': 34.019292666666665, 'long': -118.3379412},
 {'city': 'Boston', 'lat': 42.349149350000005, 'long': -71.06217327499999},
 {'city': 'Cambridge', 'lat': 42.3721995, 'long': -71.11438150000001},
 {'city': 'Philadelphia', 'lat': 39.9525762, 'long': -75.1652518},
 {'city': 'Las Vegas', 'lat': 36.1062, 'long': -115.1374},
 {'city': 'Austin', 'lat': 30.330271666666665, 'long': -97.73937033333334},
 {'city': 'Montreal', 'lat': 45.517999950000004, 'long': -73.58168965},
 {'city': 'Vancouver', 'lat': 49.28235205, 'long': -123.1088365},
 {'city': 'Chicago', 'lat': 41.889183, 'long': -87.632519},
 {'city': 'Mumbai', 'lat': 18.9742904, 'long': 72.8127265},
 {'city': 'Madrid', 'lat': 40.4273082, 'long': -3.6895115},
 {'city': 'Helsinki', 'lat': 60.171821, 'long': 24.945167}]

### 8. Hago un scraping para  añadir a cada ciudad su índice de coste de vida, me quedaré con el más bajo, creo que eso es bueno tanto para los trabajadores como para la empresa, porque ese índice incluye la adquisición de bienes y servicios (infraestructuras más baratas más mano de obra barata).

In [26]:
from getIndexCost import getLCostIndex
import pandas as pd
pd.DataFrame(getLCostIndex(cities)).sort_values(by ='liveCostIndex' ).reset_index().drop(columns = 'index').to_csv('../outputs/okByCost.csv')

In [28]:
pd.read_csv('../outputs/okByCost.csv').drop(columns = 'Unnamed: 0')

,city,lat,long,liveCostIndex
0,Mumbai,18.974290,72.812726,29.87
1,Madrid,40.427308,-3.689511,58.89
2,London,53.344104,-6.267494,59.97
3,Montreal,45.518000,-73.581690,66.47
4,Cambridge,42.372200,-71.114382,69.03
5,Austin,30.330272,-97.739370,69.17
6,Las Vegas,36.106200,-115.137400,69.37
7,Vancouver,49.282352,-123.108836,73.72
8,Helsinki,60.171821,24.945167,74.39
9,Los Angeles,34.019293,-118.337941,78.99
